# 第 10 章 数据聚合与分组运算

Pandas 提供了一个灵活高效的 **groupby** 功能，它使你能以一种自然的方式对数据集进行切片、切块、摘要等操作。

## GroupBy 机制

Hadley Wickham（许多热门R语言包的作者）创造了一个用于表示分组运算的术语"split-apply-combine"（拆分－应用－合并）。第一个阶段，pandas 对象（无论是 Series、DataFrame 还是其他的）中的数据会根据你所提供的一个或多个键被拆分（split）为多组。拆分操作是在对象的特定轴上执行的。例如，DataFrame 可以在其行（axis=0）或列（axis=1）上进行分组。然后，将一个函数应用（apply）到各个分组并产生一个新值。最后，所有这些函数的执行结果会被合并（combine）到最终的结果对象中。结果对象的形式一般取决于数据上所执行的操作。

**分组键可以有多种形式，且类型不必相同：**

- 列表或数组，其长度与待分组的轴一样  
- 表示 DataFrame 某个列名的值  
- 字典或 Series，给出待分组轴上的值与分组名之间的对应关系  
- 函数，用于处理轴索引或索引中的各个标签

In [1]:
import numpy as np
from pandas import DataFrame
import pandas as pd

df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                  'key2' : ['one', 'two', 'one', 'two', 'one'],
                  'data1' : np.random.randn(5),
                  'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-2.224250,-0.203408
1,a,two,-0.588512,0.082688
2,b,one,-1.592547,-1.933435
3,b,two,1.024331,-2.152458
4,a,one,0.678731,-1.106483


In [2]:
# 假设你想要按key1进行分组，并计算data1列的平均值。实现该功能的方式有很多，
# 而我们这里要用的是：访问data1，并根据key1调用groupby：

grouped = df['data1'].groupby(df['key1'])
grouped

In [3]:
grouped.mean()

key1
a   -0.711344
b   -0.284108
Name: data1, dtype: float64

In [4]:
# 一次传入多个数组的列表，就会得到不同的结果：
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.772760
      two    -0.588512
b     one    -1.592547
      two     1.024331
Name: data1, dtype: float64

In [5]:
means.unstack()

key2,one,two
key1,,
a,-0.772760,-0.588512
b,-1.592547,1.024331


In [6]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005   -0.588512
            2006   -1.592547
Ohio        2005   -0.599960
            2006    0.678731
Name: data1, dtype: float64

In [7]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.711344,-0.409068
b,-0.284108,-2.042946


In [8]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.772760 -0.654946
     two  -0.588512  0.082688
b    one  -1.592547 -1.933435
     two   1.024331 -2.152458

In [9]:
# GroupBy 的 size 方法，返回一个含有分组大小的 Series
# 任何分组关键词中的缺失值，都会被从结果中除去

df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 对分组进行迭代

GroupBy 对象支持迭代，可以产生一组二元元组（由分组名和数据块组成）

In [10]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)


a
  key1 key2     data1     data2
0    a  one -2.224250 -0.203408
1    a  two -0.588512  0.082688
4    a  one  0.678731 -1.106483
b
  key1 key2     data1     data2
2    b  one -1.592547 -1.933435
3    b  two  1.024331 -2.152458


In [11]:
# 对于多重键的情况，元组的第一个元素将会是由键值组成的元组：
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print(k1, k2)
    print(group)

a one
  key1 key2     data1     data2
0    a  one -2.224250 -0.203408
4    a  one  0.678731 -1.106483
a two
  key1 key2     data1     data2
1    a  two -0.588512  0.082688
b one
  key1 key2     data1     data2
2    b  one -1.592547 -1.933435
b two
  key1 key2     data1     data2
3    b  two  1.024331 -2.152458


In [12]:
# 将这些数据片段做成一个字典：
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,-1.592547,-1.933435
3,b,two,1.024331,-2.152458


In [13]:
# groupby默认是在axis=0上进行分组的，通过设置也可以在其他任何轴上进行分组。
# 拿上面例子中的df来说，我们可以根据dtype对列进行分组：
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [14]:
grouped = df.groupby(df.dtypes, axis=1)

In [15]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -2.224250 -0.203408
1 -0.588512  0.082688
2 -1.592547 -1.933435
3  1.024331 -2.152458
4  0.678731 -1.106483
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### 选取一列或列的子集

In [16]:
# 对于由DataFrame产生的GroupBy对象，如果用一个（单个字符串）或一组（字符串数组）列名对其进行索引，
# 就能实现选取部分列进行聚合的目的

df.groupby('key1')['data1']
df.groupby('key1')[['data2']]

In [17]:
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])

In [18]:
# 尤其对于大数据集，很可能只需要对部分列进行聚合。例如，在前面那个数据集中，
# 如果只需计算data2列的平均值并以DataFrame形式得到结果，可以这样写：

df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.654946
     two   0.082688
b    one  -1.933435
     two  -2.152458

In [19]:
# 这种索引操作所返回的对象是一个已分组的 DataFrame （如果传入的是列表或数组）
# 或已分组的 Series（如果传入的是标量形式的单个列名）：
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [20]:
s_grouped.mean()

key1  key2
a     one    -0.654946
      two     0.082688
b     one    -1.933435
      two    -2.152458
Name: data2, dtype: float64

### 通过字典或 Series 进行分组

In [21]:
people = pd.DataFrame(np.random.randn(5, 5),
                     columns=['a', 'b', 'c', 'd', 'e'],
                     index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # add a few NA values
people

,a,b,c,d,e
Joe,-1.414964,0.607488,0.094695,0.727992,-0.992872
Steve,-1.261667,-0.787860,1.952596,1.381011,1.049920
Wes,-0.450283,NaN,NaN,-1.382112,0.428551
Jim,0.011325,0.308478,1.747748,-0.538604,-0.360113
Travis,1.129820,-0.902642,0.897523,0.622185,1.083888


In [22]:
# 假设已知列的分组关系，并希望根据分组计算列的和：
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
          'd': 'blue', 'e': 'red', 'f': 'orange'}

In [23]:
# 可以将这个字典传给groupby，来构造数组，但我们可以直接传递字典
# （我包含了键“f”来强调，存在未使用的分组键是可以的）：

by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,0.822687,-1.800349
Steve,3.333607,-0.999608
Wes,-1.382112,-0.021732
Jim,1.209143,-0.040310
Travis,1.519707,1.311066


In [24]:
# Series也有同样的功能，它可以被看做一个固定大小的映射：
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [25]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 通过函数进行分组

比起使用字典或Series，使用Python函数是一种更原生的方法定义分组映射。任何被当做分组键的函数都会在各个索引值上被调用一次，其返回值就会被用作分组名称。具体点说，以上一小节的示例DataFrame 为例，其索引值为人的名字。你可以计算一个字符串长度的数组，更简单的方法是传入len 函数：

In [26]:
people.groupby(len).sum()

,a,b,c,d,e
3,-1.853923,0.915966,1.842443,-1.192724,-0.924435
5,-1.261667,-0.787860,1.952596,1.381011,1.049920
6,1.129820,-0.902642,0.897523,0.622185,1.083888


In [27]:
# 将函数跟数组、列表、字典、Series混合使用也不是问题，因为任何东西在内部都会被转换为数组：
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -1.414964  0.607488  0.094695 -1.382112 -0.992872
  two  0.011325  0.308478  1.747748 -0.538604 -0.360113
5 one -1.261667 -0.787860  1.952596  1.381011  1.049920
6 two  1.129820 -0.902642  0.897523  0.622185  1.083888

### 根据索引级别分组

In [28]:
# 层次化索引数据集最方便的地方就在于它能够根据轴索引的一个级别进行聚合：

columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                   names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.408236 -0.405471  0.607647  0.289101  1.394568
1      0.017218  0.628753 -0.467579 -1.295043 -0.609357
2      0.420745  1.227416  1.176447  0.645852 -1.368505
3      0.235372 -1.229477  1.392652  0.560184 -0.730345

In [29]:
# 要根据级别分组，使用level关键字传递级别序号或名字：
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 数据聚合

聚合指的是任何能够从数组产生标量值的数据转换过程

| 函数名 | 说明 |
| ------ | ---- |
| count | 分组中非 NA 值的数量 |
| sum | 非 NA 值得和 |
| mean | 非 NA 值的平均值 |
| median | 非 NA 值的算术中位数 |
| std、var | 无偏（分母为 n-1）标准差和方差 |
| min、max |非 NA 值的最小值和最大值 |
| prod | 非 NA 值的积 |
| first、last | 第一个和最后一个非 NA 值|

In [30]:
df

,key1,key2,data1,data2
0,a,one,-2.224250,-0.203408
1,a,two,-0.588512,0.082688
2,b,one,-1.592547,-1.933435
3,b,two,1.024331,-2.152458
4,a,one,0.678731,-1.106483


In [31]:
grouped = df.groupby('key1')

In [32]:
grouped['data1'].quantile(0.9)

key1
a    0.425282
b    0.762643
Name: data1, dtype: float64

In [33]:
# 如果要使用你自己的聚合函数，只需将其传入aggregate或agg方法即可：
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,2.902981,1.189171
b,2.616878,0.219022


In [34]:
# 有些方法（如describe）也是可以用在这里的，即使严格来讲，它们并非聚合运算：
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.711344  1.455383 -2.224250 -1.406381 -0.588512  0.045109   
b      2.0 -0.284108  1.850412 -1.592547 -0.938328 -0.284108  0.370111   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.678731   3.0 -0.409068  0.620688 -1.106483 -0.654946 -0.203408   
b     1.024331   2.0 -2.042946  0.154872 -2.152458 -2.097702 -2.042946   

                          
           75%       max  
key1                      
a    -0.060360  0.082688  
b    -1.988191 -1.933435

*自定义聚合函数要比表10-1中那些经过优化的函数慢得多。这是因为在
构造中间分组数据块时存在非常大的开销（函数调用、数据重排等）*

### 面向列的多函数应用

In [35]:
# 回到前面小费的例子。使用read_csv导入数据之后，我们添加了一个小费百分比的列tip_pct：
tips = pd.read_csv('tips.csv')
# add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


对 Series 或 DataFrame 列的聚合运算其实就是使用 aggregate（使用自定义函数）或调用诸如 mean、std 之类的方法。然而，你可能希望对不同的列使用不同的聚合函数，或一次应用多个函数。其实这也好办，我将通过一些示例来进行讲解。首先，我根据天和 smoker 对 tips 进行分组：

In [36]:
grouped = tips.groupby(['day', 'smoker'])

In [37]:
# 对于表10-1中的那些描述统计，可以将函数名以字符串的形式传入：
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [38]:
# 如果传入一组函数或函数名，得到的 DataFrame 的列就会以相应的函数命名：
# 这里，我们传递了一组聚合函数进行聚合，独立对数据分组进行评估。
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

你并非一定要接受 GroupBy 自动给出的那些列名，特别是 lambda 函数，它们的名称是''，这样的辨识度就很低了（通过函数的 name 属性看看就知道了）。因此，如果传入的是一个由(name,function)元组组成的列表，则各元组的第一个元素就会被用作 DataFrame 的列名（可以将这种二元元组列表看做一个有序映射）：

In [39]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

对于 DataFrame，你还有更多选择，你可以定义一组应用于全部列的一组函数，或不同的列应用不同的函数。假设我们想要对 tip_pct 和 total_bill 列计算三个统计信息：

In [40]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [41]:
# 结果DataFrame拥有层次化的列，这相当于分别对各列进行聚合，
# 然后用concat将结果组装到一起，使用列名用作keys参数：
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [42]:
# 跟前面一样，这里也可以传入带有自定义名称的一组元组：
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [43]:
# 假设你想要对一个列或不同的列应用不同的函数。
# 具体的办法是向agg传入一个从列名映射到函数的字典：
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [44]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'], 'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

*只有将多个函数应用到至少一列时，DataFrame才会拥有层次化的列。*

### 以“没有行索引”的形式返回聚合数据

到目前为止，所有示例中的聚合数据都有由唯一的分组键组成的索引（可能还是层
次化的）。由于并不总是需要如此，所以你可以向groupby传入as_index=False以
禁用该功能：

In [45]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


当然，对结果调用reset_index也能得到这种形式的结果。使用as_index=False方法
可以避免一些不必要的计算。

## apply: 一般性的“拆分--应用--合并”

apply 会将待处理的对象拆分成多个片段，然后对各片段调用传入的函数，最后尝试
将各片段组合到一起。

回到之前那个小费数据集，假设你想要根据分组选出最高的5个tip_pct值。首先，
编写一个选取指定列具有最大值的行的函数：

In [46]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


如果对 smoker 分组并用该函数调用 apply，就会得到：

In [47]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

top 函数在 DataFrame 的各个片段上调用，然后结果由pandas.concat 组装到一起，并以分组名称进行了标记。于是，最终结果就有了一个层次化索引，其内层索引值来自原 DataFrame。

如果传给 apply 的函数能够接受其他参数或关键字，则可以将这些内容放在函数名后
面一并传入：

In [48]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

在 GroupBy 对象上调用 describe

In [49]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [50]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

在 GroupBy 中，当你调用诸如 describe 之类的方法时，实际上只是应用了下面两条
代码的快捷方式而已：

In [51]:
f = lambda x: x.describe()
grouped.apply(f)

total_bill        tip       size    tip_pct
day  smoker                                                   
Fri  No     count    4.000000   4.000000   4.000000   4.000000
            mean    18.420000   2.812500   2.250000   0.151650
            std      5.059282   0.898494   0.500000   0.028123
            min     12.460000   1.500000   2.000000   0.120385
            25%     15.100000   2.625000   2.000000   0.137239
            50%     19.235000   3.125000   2.000000   0.149241
            75%     22.555000   3.312500   2.250000   0.163652
            max     22.750000   3.500000   3.000000   0.187735
     Yes    count   15.000000  15.000000  15.000000  15.000000
            mean    16.813333   2.714000   2.066667   0.174783
            std      9.086388   1.077668   0.593617   0.051293
            min      5.750000   1.000000   1.000000   0.103555
            25%     11.690000   1.960000   2.000000   0.133739
            50%     13.420000   2.500000   2.000000   0.173913
            75%     18.665000   3.240000   2.000000   0.209240
            max     40.170000   4.730000   4.000000   0.263480
Sat  No     count   45.000000  45.000000  45.000000  45.000000
            mean    19.661778   3.102889   2.555556   0.158048
            std      8.939181   1.642088   0.784960   0.039767
            min      7.250000   1.000000   1.000000   0.056797
            25%     14.730000   2.010000   2.000000   0.136240
            50%     17.820000   2.750000   2.000000   0.150152
            75%     20.650000   3.390000   3.000000   0.183915
            max     48.330000   9.000000   4.000000   0.291990
     Yes    count   42.000000  42.000000  42.000000  42.000000
            mean    21.276667   2.875476   2.476190   0.147906
            std     10.069138   1.630580   0.862161   0.061375
            min      3.070000   1.000000   1.000000   0.035638
            25%     13.405000   2.000000   2.000000   0.091797
            50%     20.390000   2.690000   2.000000   0.153624
...                       ...        ...        ...        ...
Sun  No     std      8.130189   1.224785   1.032674   0.042347
            min      8.770000   1.010000   2.000000   0.059447
            25%     14.780000   2.000000   2.000000   0.139780
            50%     18.430000   3.020000   3.000000   0.161665
            75%     25.000000   3.920000   4.000000   0.185185
            max     48.170000   6.000000   6.000000   0.252672
     Yes    count   19.000000  19.000000  19.000000  19.000000
            mean    24.120000   3.516842   2.578947   0.187250
            std     10.442511   1.261151   0.901591   0.154134
            min      7.250000   1.500000   2.000000   0.065660
            25%     17.165000   3.000000   2.000000   0.097723
            50%     23.100000   3.500000   2.000000   0.138122
            75%     32.375000   4.000000   3.000000   0.215325
            max     45.350000   6.500000   5.000000   0.710345
Thur No     count   45.000000  45.000000  45.000000  45.000000
            mean    17.113111   2.673778   2.488889   0.160298
            std      7.721728   1.282964   1.179796   0.038774
            min      7.510000   1.250000   1.000000   0.072961
            25%     11.690000   1.800000   2.000000   0.137741
            50%     15.950000   2.180000   2.000000   0.153492
            75%     20.270000   3.000000   2.000000   0.184843
            max     41.190000   6.700000   6.000000   0.266312
     Yes    count   17.000000  17.000000  17.000000  17.000000
            mean    19.190588   3.030000   2.352941   0.163863
            std      8.355149   1.113491   0.701888   0.039389
            min     10.340000   2.000000   2.000000   0.090014
            25%     13.510000   2.000000   2.000000   0.148038
            50%     16.470000   2.560000   2.000000   0.153846
            75%     19.810000   4.000000   2.000000   0.194837
            max     43.110000   5.000000   4.000000   0.241255

[64 rows x 4 columns]

### 禁止分组键

分组键会跟原始对象的索引共同构成结果对象中的层次化索引。将 group_keys=False 传入 groupby即可禁止该效果：

In [52]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### 分位数和桶分析

pandas 有一些能根据指定面元或样本分位数将数据拆分成多块的工具（比如cut和qcut）。将这些函数跟 groupby 结合起来，就能非常轻松地实现对数据集的桶（bucket）或分位数（quantile）分析了。以下面这个简单的随机数据集为例，我们利用 cut 将其装入长度相等的桶中：

In [53]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                     'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0    (-2.965, -1.298]
1     (-1.298, 0.363]
2     (-1.298, 0.363]
3     (-1.298, 0.363]
4    (-2.965, -1.298]
5     (-1.298, 0.363]
6     (-1.298, 0.363]
7      (0.363, 2.023]
8      (0.363, 2.023]
9     (-1.298, 0.363]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.965, -1.298] < (-1.298, 0.363] < (0.363, 2.023] < (2.023, 3.684]]

由cut返回的Categorical对象可直接传递到groupby。因此，我们可以像下面这样对data2列做一些统计计算：

In [54]:
def get_states(group):
    return {'min': group.min(), 'max': group.max(),
           'count': group.count(), 'mean': group.mean()}
grouped = frame.data2.groupby(quartiles)
grouped.apply(get_states).unstack()

,count,max,mean,min
data1,,,,
"(-2.965, -1.298]",97.0,2.296072,-0.103696,-2.253014
"(-1.298, 0.363]",542.0,2.773663,-0.044119,-2.738158
"(0.363, 2.023]",331.0,2.438762,-0.041314,-2.835451
"(2.023, 3.684]",30.0,2.282679,0.186260,-1.731386


这些都是长度相等的桶。要根据样本分位数得到大小相等的桶，使用 qcut 即可。传入 labels=False 即可只获取分位数的编号：

In [55]:
# Return quantile numbers
grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_states).unstack()

,count,max,mean,min
data1,,,,
0,100.0,2.296072,-0.131088,-2.253014
1,100.0,2.170551,0.041416,-2.638588
2,100.0,2.504957,0.009771,-2.454738
3,100.0,1.869657,0.025542,-2.408903
4,100.0,2.662174,-0.036572,-2.283154
5,100.0,2.773663,-0.126621,-2.211342
6,100.0,2.390726,-0.229738,-2.738158
7,100.0,2.438762,-0.084626,-2.835451
8,100.0,2.375955,-0.040586,-2.495955


### 示例：用特定于分组的值填充缺失值

In [56]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1   -0.418493
2         NaN
3   -1.396308
4         NaN
5   -1.662065
dtype: float64

In [57]:
s.fillna(s.mean())

0   -1.158955
1   -0.418493
2   -1.158955
3   -1.396308
4   -1.158955
5   -1.662065
dtype: float64

**假设你需要对不同的分组填充不同的值。一种方法是将数据分组，并使用apply和一
个能够对各数据块调用fillna的函数即可。**

In [61]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
         'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data
# ['East'] * 4产生了一个列表，包括了['East']中元素的四个拷贝。将这些列表串联起来

Ohio         -0.285498
New York      1.132737
Vermont      -1.055980
Florida       0.212801
Oregon       -1.054092
Nevada        1.332216
California   -0.025636
Idaho        -0.906593
dtype: float64

In [59]:
# 将一些值设为缺失：
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio          1.156667
New York     -0.192010
Vermont            NaN
Florida      -0.583511
Oregon        0.157831
Nevada             NaN
California    1.187246
Idaho              NaN
dtype: float64

In [60]:
data.groupby(group_key).mean()

East    0.127049
West    0.672538
dtype: float64

In [62]:
# 我们可以用分组平均值去填充NA值:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio         -0.285498
New York      1.132737
Vermont      -1.055980
Florida       0.212801
Oregon       -1.054092
Nevada        1.332216
California   -0.025636
Idaho        -0.906593
dtype: float64

In [63]:
# 另外，也可以在代码中预定义各组的填充值。由于分组具有一个name属性，
# 所以我们可以拿来用一下：
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio         -0.285498
New York      1.132737
Vermont      -1.055980
Florida       0.212801
Oregon       -1.054092
Nevada        1.332216
California   -0.025636
Idaho        -0.906593
dtype: float64

### 示例：随机采样和排列

**假设你想要从一个大数据集中随机抽取（进行替换或不替换）样本以进行蒙特卡罗
模拟（Monte Carlo simulation）或其他分析工作。“抽取”的方式有很多，这里使用
的方法是对Series使用sample方法：**

In [64]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)
deck = pd.Series(card_val, index=cards)

**现在我有了一个长度为52的Series，其索引包括牌名，值则是21点或其他游戏中用
于计分的点数（为了简单起见，我当A的点数为1）：**

In [65]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [66]:
# 现在，根据我上面所讲的，从整副牌中抽出5张，代码如下：
def draw(deck, n=5):
    return deck.sample(n)
draw(deck)

5C    5
AC    1
9D    9
4D    4
5S    5
dtype: int64

假设你想要从每种花色中随机抽取两张牌。由于花色是牌名的最后一个字符，所以
我们可以据此进行分组，并使用apply：

In [67]:
get_suit = lambda card: card[-1] # last letter is suit
deck.groupby(get_suit).apply(draw, n=2)

C  5C     5
   QC    10
D  5D     5
   JD    10
H  4H     4
   QH    10
S  5S     5
   4S     4
dtype: int64

In [68]:
# 或者，也可以这样写：
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

3C     3
KC    10
QD    10
AD     1
KH    10
AH     1
2S     2
3S     3
dtype: int64

### 示例：分组加权平均数和相关系数

根据groupby的“拆分－应用－合并”范式，可以进行DataFrame的列与列之间或两个
Series之间的运算（比如分组加权平均）。以下面这个数据集为例，它含有分组
键、值以及一些权重值：

In [69]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                               'b', 'b', 'b', 'b'],
                  'data': np.random.randn(8),
                  'weights': np.random.rand(8)})
df

,category,data,weights
0,a,0.164166,0.935750
1,a,-0.571535,0.588652
2,a,1.414772,0.496113
3,a,0.955825,0.097663
4,b,1.056315,0.079676
5,b,-0.793813,0.783562
6,b,0.116646,0.461615
7,b,-1.139013,0.406528


In [70]:
# 然后可以利用category计算分组加权平均数：
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

category
a    0.289125
b   -0.546982
dtype: float64

**另一个例子，考虑一个来自Yahoo!Finance的数据集，其中含有几只股票和标准普
尔500指数（符号SPX）的收盘价：**

In [72]:
close_px = pd.read_csv('stock_px_2.csv', parse_dates=True, index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


In [73]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


**来做一个比较有趣的任务：计算一个由日收益率（通过百分数变化计算）与SPX之
间的年度相关系数组成的DataFrame。下面是一个实现办法，我们先创建一个函
数，用它计算每列和SPX列的成对相关系数：**

In [74]:
spx_corr = lambda x: x.corrwith(x['SPX'])

**接下来，我们使用pct_change计算close_px的百分比变化：**

In [75]:
rets = close_px.pct_change().dropna()

**最后，我们用年对百分比变化进行分组，可以用一个一行的函数，从每行的标签返
回每个datetime标签的year属性：**

In [76]:
get_year = lambda x: x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


当然，你还可以计算列与列之间的相关系数。这里，我们计算Apple和Microsoft的
年相关系数：

In [77]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### 示例：组级别的线性回归

顺着上一个例子继续，你可以用groupby执行更为复杂的分组统计分析，只要函数
返回的是pandas对象或标量值即可。例如，我可以定义下面这个regress函数（利
用statsmodels计量经济学库）对各数据块执行普通最小二乘法（Ordinary Least
Squares，OLS）回归：

In [79]:
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

现在，为了按年计算AAPL对SPX收益率的线性回归，执行：

In [80]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## 透视表和交叉表

透视表（pivot table）是各种电子表格程序和其他数据分析软件中一种常见的数据
汇总工具。它根据一个或多个键对数据进行聚合，并根据行和列上的分组键将数据
分配到各个矩形区域中。在Python和pandas中，可以通过本章所介绍的groupby功
能以及（能够利用层次化索引的）重塑运算制作透视表。DataFrame有一个
pivot_table方法，此外还有一个顶级的pandas.pivot_table函数。除能为groupby提
供便利之外，pivot_table还可以添加分项小计，也叫做margins。

回到小费数据集，假设我想要根据day和smoker计算分组平均数（pivot_table的默
认聚合类型），并将day和smoker放到行上：

In [81]:
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

可以用groupby直接来做。现在，假设我们只想聚合tip_pct和size，而且想根据time
进行分组。我将smoker放到列上，把day放到行上：

In [82]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

还可以对这个表作进一步的处理，传入margins=True添加分项小计。这将会添加标
签为All的行和列，其值对应于单个等级中所有数据的分组统计：

In [83]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], 
                 columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

这里，All值为平均数：不单独考虑烟民与非烟民（All列），不单独考虑行分组两个
级别中的任何单项（All行）。

要使用其他的聚合函数，将其传给aggfunc即可。例如，使用count或len可以得到有
关分组大小的交叉表（计数或频率）：

In [84]:
tips.pivot_table('tip_pct', index=['time', 'smoker'],
                columns='day', aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

如果存在空的组合（也就是NA），你可能会希望设置一个fill_value：

In [87]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

### 交叉表： crosstab

交叉表（cross-tabulation，简称crosstab）是一种用于计算分组频率的特殊透视
表。看下面的例子：

In [96]:
from io import StringIO
data = """\
Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_csv(StringIO(data), sep='\s+')

In [92]:
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


作为调查分析的一部分，我们可能想要根据国籍和用手习惯对这段数据进行统计汇
总。虽然可以用pivot_table实现该功能，但是pandas.crosstab函数会更方便：

In [94]:
pd.crosstab(data.Nationality, data.Handedness, margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


crosstab的前两个参数可以是数组或Series，或是数组列表。就像小费数据：

In [95]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244